## Section 1. Import Libraries and Verify API key is Found

In [2]:
# Standard library imports
import os
import sys
import time # to pause between API calls

# Third-party imports
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
import sklearn
from urllib3.util.retry import Retry


# look for a .env file
load_dotenv()

# get the key
CENSUS_API_KEY = os.getenv("CENSUS_API_KEY")
# verify we have it
print("Key loaded:", bool(CENSUS_API_KEY))

# ensure a data directory exists
os.makedirs("data", exist_ok=True)


Key loaded: True


---
## Section 2: Load Data

### 2.1 Gini Index
#### API Query Parameters

The `params` dictionary builds the request to the Census API:

* **`get`**: Specifies the variables we want to download:
    * `NAME`: The state's name.
    * `B19083_001E`: Gini Index of Income Inequality (the estimate).
    * `B19083_001M`: The Margin of Error for the Gini Index.
* **`for`**: Sets the geographic scope. `state:*` means "get data for all states."
* **`key`**: Authenticates the request using your personal API key.

In [3]:
params = {
    "get": "NAME,B19083_001E,B19083_001M",
    "for": "state:*",
    "key": CENSUS_API_KEY  # safely injected
}

#### Build and Clean the DataFrame

Convert the raw API rows into a clean Dataframe
- Create a DataFrame with the collected rows and captured cols.
- Rename Census codes to friendly names: gini (estimate) and gini_moe (margin of error).
- Convert numeric columns from strings to numbers.
- Normalize the state code to a 2-digit FIPS string (state_fips).
- Drop duplicate state column, sort by year and state_fips, and reset the index.
- Print the final shape and list any years that failed.
- Save the dataset to a CSV file for later analysis.